In [2]:
import numpy as np
import numpy.matlib
from matplotlib import pyplot as plt
import math
import time
from ucimlrepo import fetch_ucirepo, list_available_datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [3]:
list_available_datasets()

# fetch the five different datasets from UCI 
adult = fetch_ucirepo(id=2)
flags = fetch_ucirepo(id=39)
iris = fetch_ucirepo(id=52)
mushroom = fetch_ucirepo(id=73)
wine = fetch_ucirepo(id=109)

-------------------------------------
The following datasets are available:
-------------------------------------
Dataset Name                                                                            ID    
------------                                                                            --    
Abalone                                                                                 1     
Adult                                                                                   2     
Annealing                                                                               3     
Audiology (Standardized)                                                                8     
Auto MPG                                                                                9     
Automobile                                                                              10    
Balance Scale                                                                           12    
Balloons                       

## Pre-processing

In [4]:
def preprocessing(X, y):

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_encoded = pd.get_dummies(X, drop_first=True)

    # split the data 70-30 
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size = 0.2, random_state=2)

    num_col = X.select_dtypes(include=['int64','float64']).columns

    if len(num_col) > 0:
        scaler = StandardScaler()
        X_train[num_col] = scaler.fit_transform(X_train[num_col])
        X_test[num_col] = scaler.transform(X_test[num_col])

    return X_train, X_test, y_train, y_test

In [5]:
adult_train, adult_test, adult_label_train, adult_label_test = preprocessing(adult.data.features, adult.data.targets)
flags_train, flags_test, flags_label_train, flags_label_test = preprocessing(flags.data.features, flags.data.targets)
iris_train, iris_test, iris_label_train, iris_label_test = preprocessing(iris.data.features, iris.data.targets)
mushroom_train, mushroom_test, mushroom_label_train, mushroom_label_test = preprocessing(mushroom.data.features, mushroom.data.targets)
wine_train, wine_test, wine_label_train, wine_label_test = preprocessing(wine.data.features, wine.data.targets)

NameError: name 'ada_preprocessing' is not defined

## AdaBoost, supervised learning 

In [131]:
def adaBoostpred(X_train, X_test, y_train, y_test, set_name):

    clf = AdaBoostClassifier(n_estimators=100, random_state=2)
    clf.fit(X_train, y_train)
    clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("AdaBoost accuracy for the ", set_name, "dataset: ", acc)

    return acc 



In [132]:
sets = ['adult', 'flags', 'iris', 'mushroom', 'wine']
tot_acc = 0
for name in  sets: 
    # Dynamically retrieve the variables for train/test data
    X_train = globals()[f"{name}_train"]
    X_test = globals()[f"{name}_test"]
    y_train = globals()[f"{name}_label_train"]
    y_test = globals()[f"{name}_label_test"]
    
    # Call the function with dynamically retrieved data
    acc = adaBoostpred(X_train, X_test, y_train, y_test, name)
    tot_acc = tot_acc + acc 

avarage_acc = tot_acc / len(sets)

print("Avarage accuracy for AdaBoost: ", avarage_acc)

AdaBoost accuracy for the  adult dataset:  0.5639800723401351
AdaBoost accuracy for the  flags dataset:  0.7524752475247525
AdaBoost accuracy for the  iris dataset:  0.9056603773584906
AdaBoost accuracy for the  mushroom dataset:  0.9983593109105825
AdaBoost accuracy for the  wine dataset:  0.8888888888888888
Avarage accuracy for AdaBoost:  0.8218727794045698


## K-Means Clustering 

In [133]:
def kMeanspred(X_train, X_test, y_train, y_test, set_name):
    clf = KMeans(n_clusters=len(np.unique(y_train)), random_state=12)
    clf.fit(X_train)
    y_pred = clf.predict(X_test)

    mapping ={}

    for cluster in np.unique(y_pred):
        true_labels = y_test[y_pred == cluster]
        most_common_label = np.bincount(true_labels).argmax()
        mapping[cluster] = most_common_label
    y_mapped = [mapping[cluster] for cluster in y_pred]

    acc = accuracy_score(y_test, y_mapped)
    #print(clf.cluster_centers_)
    print("K-Means accuracy for the ", set_name, " dataset: ", acc) 

    return acc

In [134]:
sets = ['adult', 'flags', 'iris', 'mushroom', 'wine']
tot_acc = 0
for name in  sets: 
    # Dynamically retrieve the variables for train/test data
    X_train = globals()[f"{name}_train"]
    X_test = globals()[f"{name}_test"]
    y_train = globals()[f"{name}_label_train"]
    y_test = globals()[f"{name}_label_test"]
    
    # Call the function with dynamically retrieved data
    acc = kMeanspred(X_train, X_test, y_train, y_test, name)
    tot_acc = tot_acc + acc 

avarage_acc = tot_acc / len(sets)

print("Avarage accuracy for K-Means: ", avarage_acc)

K-Means accuracy for the  adult  dataset:  0.5048113014399782
K-Means accuracy for the  flags  dataset:  0.7623762376237624
K-Means accuracy for the  iris  dataset:  0.7169811320754716
K-Means accuracy for the  mushroom  dataset:  0.8913043478260869
K-Means accuracy for the  wine  dataset:  1.0
Avarage accuracy for K-Means:  0.7750946037930598


In [135]:
print(

TypeError: 'tuple' object is not callable